In [1]:
import scipy.misc
import time

import tensorflow as tf
import numpy as np
import vizdoom as vd

from tqdm import trange
from IPython.display import HTML


In [2]:
#Specify the game scenario and the screen format/resolution

game = vd.DoomGame()
game.set_screen_format(vd.ScreenFormat.BGR24)
game.set_screen_resolution(vd.ScreenResolution.RES_640X480)
game.load_config('rocket_basic.cfg')

down_sample_ratio = 0.125
width = int(game.get_screen_width()*down_sample_ratio)
height = int(game.get_screen_height()*down_sample_ratio)
channels = game.get_screen_channels()

#Specify the available actions in the scenario

left = [1, 0, 0]
right = [0, 1, 0]
#shoot = [0, 0, 1]
actions = [left, right, shoot]
num_actions = len(actions)

#Specify the Q-network learning parameters

frame_delay = 12
buffer_size = 10000
epochs = 30
steps_per_epoch = 2000
discount_factor = 0.99
learning_rate = 0.001
start_epsilon = 1.0
end_epsilon = 0.1
batch_size = 100
load_model = True
save_model = True
model_dir = './checkpoints/rocket_basic.ckpt'


NameError: name 'shoot' is not defined

In [4]:
#Create a buffer object that holds a set of training experiences (state-action-reward tuples)

class Buffer():
    def __init__(self, size=1000):
        self.buffer = list()
        self.length = len(self.buffer)
        self.size = size
        
#Add new experiences to the buffer (remove old experiences if necessary to avoid exceeding the buffer size)
        
    def add_experience(self, experience):
        if self.length + 1 >= self.size:
            self.buffer[0:(self.length + 1) - self.size] = []
        
        self.buffer.append(experience)
        self.length = len(self.buffer)
            
#Return a batch of experience arrays randomly sampled from the buffer
            
    def sample_buffer(self, sample_size):
        sample = np.random.randint(self.length, size=sample_size)
        s1 = np.concatenate([self.buffer[idx][0] for idx in sample], axis=0)
        a = np.array([self.buffer[idx][1] for idx in sample])
        r = np.array([self.buffer[idx][2] for idx in sample])
        s2 = np.concatenate([self.buffer[idx][3] for idx in sample], axis=0)
        terminal = np.array([self.buffer[idx][4] for idx in sample], dtype=np.int32)
        
        return s1, a, r, s2, terminal

#Downsample and normalize an image array representing the game state at a given time stamp

def preprocess(image, down_sample_ratio=1):
    if down_sample_ratio != 1:
        image = scipy.misc.imresize(image, down_sample_ratio)
    image = image.astype(np.float32)
    image /= 255.0
    image = np.expand_dims(image, axis=0)
    
    return image
    

In [4]:
#Create a Q-network to estimate values and choose actions for a given state

tf.reset_default_graph()

s_t = tf.placeholder(tf.float32, shape=[None, height, width, channels], name='state')
a_t = tf.placeholder(tf.int32, shape=[None], name='action')
Q_target = tf.placeholder(tf.float32, shape=[None, num_actions], name='Q_target')

input_layer = tf.reshape(s_t, [-1, height, width, channels], name='input_layer')
conv1 = tf.layers.conv2d(inputs=input_layer,
                         filters=32,
                         kernel_size=[8, 8],
                         strides=[4, 4],
                         padding='valid',
                         activation=tf.nn.relu,
                         name='conv1_layer')
conv2 = tf.layers.conv2d(inputs=conv1,
                         filters=64,
                         kernel_size=[4, 4],
                         strides=[2, 2],
                         padding='valid',
                         activation=tf.nn.relu,
                         name='conv2_layer')
flatten = tf.reshape(conv2, [-1, 6*8*64], name='flatten')
dense1 = tf.layers.dense(inputs=flatten,
                         units=512,
                         activation=tf.nn.relu,
                         name='dense1_layer')
Q_values = tf.layers.dense(inputs=dense1,
                           units=len(actions),
                           activation=None,
                           name='output_layer')        
    
best_action = tf.argmax(Q_values, 1)
loss = tf.losses.mean_squared_error(Q_values, Q_target)
adam = tf.train.AdamOptimizer(learning_rate=learning_rate, name='adam').minimize(loss)

def calculate_loss(session, s, q):
    L, _ = session.run([loss, adam], feed_dict={s_t: s, Q_target: q})
    
    return L

#Return the array of Q-values and the best action associated with a given state

def get_Q_values(session, s):
    Q = session.run(Q_values, feed_dict={s_t: s})

    return Q
    
def choose_action(session, s):
    a = session.run(best_action, feed_dict={s_t: s})
    
    return a
    

In [5]:
#Play the game by choosing random actions drawn from a uniform distribution to act as a baseline example

game.set_sound_enabled(True)
game.init()
episode_rewards = list()

for i in range(20):
    game.new_episode()
    
    while not game.is_episode_finished():
        action  = np.random.randint(num_actions)
        reward = game.make_action(actions[action])
        
#Insert a 0.02 second delay after each time step so that the episode is played at normal speed
        
        time.sleep(0.02)
    
    episode_rewards.append(game.get_total_reward())
    print('Random Episode {} Reward: {}'.format(i + 1, game.get_total_reward()))
    time.sleep(1)
    
game.close()
print('Average Random Reward:', np.mean(episode_rewards))


Random Episode 1 Reward: 73.0
Random Episode 2 Reward: -91.0
Random Episode 3 Reward: 67.0
Random Episode 4 Reward: 75.0
Random Episode 5 Reward: 75.0
Random Episode 6 Reward: -370.0
Random Episode 7 Reward: -33.0
Random Episode 8 Reward: 72.0
Random Episode 9 Reward: 18.0
Random Episode 10 Reward: 73.0
Random Episode 11 Reward: 68.0
Random Episode 12 Reward: 69.0
Random Episode 13 Reward: 73.0
Random Episode 14 Reward: -370.0
Random Episode 15 Reward: -117.0
Random Episode 16 Reward: -365.0
Random Episode 17 Reward: -370.0
Random Episode 18 Reward: 49.0
Random Episode 19 Reward: 75.0
Random Episode 20 Reward: -370.0
Average Random Reward: -64.95


In [2]:
#Embed a recording of the untrained agent playing 20 episodes

HTML('<iframe src="https://drive.google.com/file/d/13MuEOfy6uY6LxSLg4L0Xsi2hjx5qnsKt/preview" width="640" height="480"></iframe>')


In [7]:
#For each time step, collect the following data:
#The current game state
#The action that was taken taken
#The reward obtained from the chosen action
#The next game state (store the first game state if the previous action ends the episode)
#A variable indicating whether the episode is over yet


exp_buffer = Buffer(size=buffer_size)
session = tf.Session()
session.run(tf.global_variables_initializer())

game.set_sound_enabled(False)
game.init()
t = 0

#Accumulate experiences in the buffer using an epsilon-greedy strategy with three training phases

for epoch in range(epochs):
    epoch_rewards = list()
    
    for step in trange(steps_per_epoch, leave=True):
        experience = list()
        game.new_episode()
        
        while not game.is_episode_finished():
            state = game.get_state()
            state1 = preprocess(state.screen_buffer, down_sample_ratio)
            
#Explore the environment by choosing random actions with 100% probability for the first phase of training

            if epoch < 0.2*epochs:
                action = np.random.randint(num_actions)
            
#Increase the probability of greedily choosing an action by a constant amount at each epoch in the second phase
            
            elif epoch < 0.9*epochs:
                epsilon = start_epsilon - (epoch + 1 - 0.2*epochs)*(start_epsilon-end_epsilon)/(0.7*epochs)
            
                if np.random.uniform(0, 1) <= epsilon:
                    action = np.random.randint(num_actions)
                
                else:
                    action = choose_action(session, state1)[0]

#Select a random action with 10% probability in the final phase of training
                
            else:
                if np.random.uniform(0, 1) <= end_epsilon:
                    action = np.random.randint(num_actions)
                    
                else:
                    action = choose_action(session, state1)[0]

            reward = game.make_action(actions[action], frame_delay)
            done = game.is_episode_finished()
            
            if done == False:
                state = game.get_state()
                state2 = preprocess(state.screen_buffer, down_sample_ratio)
        
            elif done == True:
                state2 = state1
        
#Add the experience obtained from each time step to the buffer

            t += 1
            exp_buffer.add_experience((state1, action, reward, state2, done))
        
#Sample a minibatch from the buffer if there are enough experiences in the buffer

        if exp_buffer.length > batch_size:
            s1, a, r, s2, terminal = exp_buffer.sample_buffer(batch_size)
            
#Train the Q-network by using the minibatch to update the action-value function Q
            
            Q2 = np.max(get_Q_values(session, s2), axis=1)
            target_Q = get_Q_values(session, s1)
            target_Q[np.arange(batch_size), a] = r + discount_factor*(1 - terminal)*Q2
            calculate_loss(session, s1, target_Q)
            
        epoch_rewards.append(game.get_total_reward())
        
    print('Epoch {} Mean Reward: {}'.format(epoch + 1, np.mean(epoch_rewards)))
        
print('{} time steps experienced during training'.format(t))
game.close()

if save_model == True:
    print('Model saved to', model_dir)
    tf.train.Saver().save(session, model_dir)


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [08:46<00:00,  3.80it/s]


Epoch 1 Mean Reward: -197.774


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [08:41<00:00,  3.83it/s]


Epoch 2 Mean Reward: -195.903


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [08:15<00:00,  4.04it/s]


...

Epoch 28 Mean Reward: 49.6965


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [02:35<00:00, 12.90it/s]


Epoch 29 Mean Reward: 50.028


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [02:30<00:00, 13.33it/s]


Epoch 30 Mean Reward: 49.849
639697 time steps experienced during training
Model saved to ./checkpoints/basic.ckpt


In [12]:
#Test the fully trained model by only choosing actions with a greedy strategy

if load_model == True:
    session = tf.Session()
    print('Loading model from', model_dir)
    tf.train.Saver().restore(session, model_dir)

game.set_sound_enabled(True)
game.init()
episode_rewards = list()

for i in range(20):
    game.new_episode()
    
    while not game.is_episode_finished():
        state = game.get_state()
        state1 = preprocess(state.screen_buffer, down_sample_ratio)
        action = choose_action(session, state1)[0]
        reward = game.make_action(actions[action])
        time.sleep(0.02)
        
    episode_rewards.append(game.get_total_reward())
    print('Test Episode {} Reward: {}'.format(i + 1, game.get_total_reward()))
    time.sleep(1)
    
game.close()
print('Average Test Reward:', np.mean(episode_rewards))


Loading model from ./checkpoints/rocket_basic.ckpt
INFO:tensorflow:Restoring parameters from ./checkpoints/rocket_basic.ckpt
Test Episode 1 Reward: 71.0
Test Episode 2 Reward: 71.0
Test Episode 3 Reward: 71.0
Test Episode 4 Reward: 71.0
Test Episode 5 Reward: 69.0
Test Episode 6 Reward: 71.0
Test Episode 7 Reward: 71.0
Test Episode 8 Reward: 71.0
Test Episode 9 Reward: 71.0
Test Episode 10 Reward: 71.0
Test Episode 11 Reward: 71.0
Test Episode 12 Reward: 71.0
Test Episode 13 Reward: 56.0
Test Episode 14 Reward: 71.0
Test Episode 15 Reward: 71.0
Test Episode 16 Reward: 71.0
Test Episode 17 Reward: 71.0
Test Episode 18 Reward: 71.0
Test Episode 19 Reward: 30.0
Test Episode 20 Reward: 71.0
Average Test Reward: 68.1


In [6]:
#Embed a recording of the fully trained agent playing 20 episodes

HTML('<iframe src="https://drive.google.com/file/d/1nbnPwtomrITlh-QdSXCPA6S85RlOlVzW/preview" width="640" height="480"></iframe>')
